<img style="float: right" src="img/saturn.png" width="300" />

# Scaling Machine Learning in Python

## Single-node workflow

We'll first start off with a typical data preparation and machine learning workflow utilizing only the Jupyter Server.


## Monitor resource utilization

For this workshop it's important to monitor CPU and memory utilization when running various commands. It will help with understanding which operations are slow - and which ones run faster on a cluster!

To monitor resource utilization of the Jupyter Server, open a new Terminal window inside Jupyter Lab and run `htop`. You can position the window to view the notebook and terminal on the same screen:

![htop](img/htop.png)

# Load data

This workshop will utilize data for yellow taxi rides from the 2019 calendar year. The machine learning exercises involve predicting the "tip fraction" of each ride - how much a rider will tip the driver as a fraction of the charged fare amount.

Let's operate with one month for now to explore the data and build out the machine learning code.

> Note: `s3.open()` is only required because we're using an anonymous S3 connection. If you have AWS credentials set up, you could pass in just the S3 URL string to `pd.read_csv()`.

In [1]:
import s3fs
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

s3 = s3fs.S3FileSystem(anon=True)

In [2]:
jan2019_file = 's3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv'
jan2019_size = s3.du(jan2019_file)
print(f"Size: {round(jan2019_size / 1e6, 2)} MB")

Size: 687.09 MB


In [3]:
%%time

taxi = pd.read_csv(
    s3.open(jan2019_file, mode='rb',),
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
)

CPU times: user 13.3 s, sys: 3.24 s, total: 16.5 s
Wall time: 34.9 s


In [4]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


### Exercise

How many rows are in the `taxi` DataFrame?

In [ ]:
<FILL IN>

In [ ]:
len(taxi)

Memory usage is also an important consideration, as DataFrames often take more space in memory than on disk:

In [6]:
taxi_bytes = taxi.memory_usage(deep=True).sum()
print(f"Size (MB): {taxi_bytes / 1e6}")

Size (MB): 1487.551776


# Exploratory analysis

For this workshop, we will just look at column statistics. There are many more explorary analyses that can performed with `pandas` and data visualization tools

In [7]:
%%time
np.round(taxi.describe().T, 3)

CPU times: user 3.87 s, sys: 291 ms, total: 4.16 s
Wall time: 4.16 s


,count,mean,std,min,25%,50%,75%,max
VendorID,7667792.0,1.637,0.540,1.0,1.00,2.00,2.00,4.00
passenger_count,7667792.0,1.567,1.224,0.0,1.00,1.00,2.00,9.00
trip_distance,7667792.0,2.801,3.738,0.0,0.90,1.53,2.80,831.80
RatecodeID,7667792.0,1.058,0.678,1.0,1.00,1.00,1.00,99.00
PULocationID,7667792.0,165.501,66.392,1.0,130.00,162.00,234.00,265.00
DOLocationID,7667792.0,163.753,70.364,1.0,113.00,162.00,234.00,265.00
payment_type,7667792.0,1.292,0.473,1.0,1.00,1.00,2.00,4.00
fare_amount,7667792.0,12.409,262.072,-362.0,6.00,8.50,13.50,623259.86
extra,7667792.0,0.328,0.507,-60.0,0.00,0.00,0.50,535.38
mta_tax,7667792.0,0.497,0.053,-0.5,0.50,0.50,0.50,60.80


# Feature engineering

We are using stateless features, meaning the features values for a given observation don't depend on other observations. This is allows us to create features before performing any data splitting.

Then, split data into train/test sets.

In [7]:
# specify feature and label column names
raw_features = [
    'tpep_pickup_datetime', 
    'passenger_count', 
    'tip_amount', 
    'fare_amount',
]
features = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
label = 'tip_fraction'

In [8]:
def prep_df(taxi_df):
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = taxi_df[taxi_df.fare_amount > 0][raw_features].copy()  # avoid divide-by-zero
    df[label] = df.tip_amount / df.fare_amount
     
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [label]].astype(float).fillna(-1)
    
    return df

In [9]:
taxi_feat = prep_df(taxi)
taxi_feat.head()

,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,tip_fraction
0,1.0,1.0,0.0,24.0,46.0,1.0,0.235714
1,1.0,1.0,0.0,24.0,59.0,1.0,0.071429
2,4.0,51.0,13.0,109.0,48.0,3.0,0.000000
3,2.0,48.0,15.0,63.0,52.0,5.0,0.000000
4,2.0,48.0,15.0,63.0,56.0,5.0,0.000000


Split into train/test sets

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    taxi_feat[features], 
    taxi_feat[label], 
    test_size=0.3,
    random_state=42
)

In [11]:
X_train.shape, y_train.shape

((5360764, 6), (5360764,))

In [12]:
X_test.shape, y_test.shape

((2297471, 6), (2297471,))

# Train model

We'll train a linear model to predict `tip_fraction`. We define a `Pipeline` to encompass both feature scaling and model training. This will be useful later when performing a grid search.

Evaluate the model against the test set using RMSE. We'll also save out the model for later use.

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

pipeline = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('clf', ElasticNet(normalize=False, max_iter=100, l1_ratio=0)),
])

In [14]:
%%time
fitted = pipeline.fit(X_train, y_train)

CPU times: user 13.9 s, sys: 240 ms, total: 14.2 s
Wall time: 7.58 s


In [15]:
%%time
preds = fitted.predict(X_test)

CPU times: user 162 ms, sys: 23.4 ms, total: 185 ms
Wall time: 92 ms


In [16]:
mean_squared_error(y_test, preds, squared=False)

5.118045186129104

In [17]:
import cloudpickle

with open('/tmp/model.pkl', 'wb') as f:
    cloudpickle.dump(fitted, f)

## Hooray!

We trained a terrible model. But that's okay. The point of this workshop is to scale our work, not make the model perfect!

# Let's step things up

We were able to train a single, simple model on a sample of the taxi data (single month from 2019). In most model building settings more data and more compute would be required. We will explore the following scenarios where the single-node environment has challenges - and see how Dask solves these problems!

- Tune hyperparameters (train numerous models)
- Load and process large dataset (full year's data)
- Predict over large dataset
- Train model with large dataset

## Tune hyperparameters

Let's stick to the single month of data and try a hyperparameter search. One model can be trained comfortably and fairly quickly with the amount of memory and processing power the Jupyter Server has. A hyperparameter search involves training many models - which means it needs a lot of processing power to finish in a reasonable amount of time.

Scikit-learn's `GridSearchCV` accepts an `n_jobs` argument, which will train `n` different models at the same time. Setting it to `-1` uses all the CPU cores in this Jupyter Server. Watch `htop` here - the cores will be saturated while memory will not be.

In [49]:
from sklearn.model_selection import GridSearchCV

params = {
    'clf__l1_ratio': np.arange(0, 1.1, 0.1),
    'clf__alpha': [0, 0.5, 1, 2],
}

grid_search = GridSearchCV(
    pipeline, 
    params, 
    cv=3, 
    n_jobs=-1, 
    verbose=1, 
    scoring='neg_mean_squared_error',
)

In [50]:
%%time
_ = grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 44 candidates, totalling 132 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 132 out of 132 | elapsed:  4.5min finished


CPU times: user 3.1 s, sys: 3.6 s, total: 6.7 s
Wall time: 4min 31s


-79.51625758585993

### This works, but...

This is a pretty small grid search. With a larger grid, it would take quite a while.

![waiting](https://media3.giphy.com/media/QBd2kLB5qDmysEXre9/giphy.gif?cid=ecf05e47t2wdryi59apjgwa3okzhxpctvxjsqaxautawn96i&rid=giphy.gif)

--- 

## Load and process large dataset

Let's look at the size of the files on disk using `s3fs`. 

In [55]:
files = s3.glob('s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv')
total_size = 0
for f in files:
    size = s3.du(f)
    total_size += size
    
    print(f"{f}, Size: {round(size / 1e6, 2)} MB")
print()
print(f"Total size: {round(total_size / 1e9, 2)} GB")

nyc-tlc/trip data/yellow_tripdata_2019-01.csv, Size: 687.09 MB
nyc-tlc/trip data/yellow_tripdata_2019-02.csv, Size: 649.88 MB
nyc-tlc/trip data/yellow_tripdata_2019-03.csv, Size: 726.2 MB
nyc-tlc/trip data/yellow_tripdata_2019-04.csv, Size: 689.21 MB
nyc-tlc/trip data/yellow_tripdata_2019-05.csv, Size: 701.54 MB
nyc-tlc/trip data/yellow_tripdata_2019-06.csv, Size: 643.49 MB
nyc-tlc/trip data/yellow_tripdata_2019-07.csv, Size: 584.39 MB
nyc-tlc/trip data/yellow_tripdata_2019-08.csv, Size: 562.39 MB
nyc-tlc/trip data/yellow_tripdata_2019-09.csv, Size: 608.97 MB
nyc-tlc/trip data/yellow_tripdata_2019-10.csv, Size: 669.17 MB
nyc-tlc/trip data/yellow_tripdata_2019-11.csv, Size: 637.81 MB
nyc-tlc/trip data/yellow_tripdata_2019-12.csv, Size: 639.11 MB

Total size: 7.8 GB


<br>

Files end up taking more space when loaded into memory, but let's see if this will fit on our Jupyter Server. We can loop through the files and concatenate them into one DataFrame. Watch memory utilization as the loop runs!

In [56]:
def load_csv(file):
    df = pd.read_csv(
        s3.open(file, mode='rb'),
        parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
    )
    return df

In [ ]:
%%time

dfs = []
for f in files:
    print(f)
    df = load_csv(f)
    print(f'{len(df)} rows, {df.memory_usage(deep=True).sum() / 1e6} MB')
    dfs.append(df)
taxi_big = pd.concat(dfs)

![explosion](https://media4.giphy.com/media/13d2jHlSlxklVe/giphy.gif)

Oh no! Looks like we have enough memory to load the CSV files individually, but not to concatenate them together into one DataFrame.

--- 

## Predict over large dataset

We can't load a large dataset into one DataFrame, but if we needed to predict over a large dataset we could batch it and collect the results.

> Note: If your kernel died when trying to concatenate the big DataFrame, you will need to re-run all the preceding cells in this notebook - just not the one that killed the kernel!

This will work, but watch `htop` - only one core will be used at a time.

In [64]:
%%time

predicted_dfs = []
for f in files:
    print(f)
    df = load_csv(f)
    feat_df = prep_df(df)
    predicted = fitted.predict(feat_df[features])
    predicted_df = pd.DataFrame(predicted, columns=['prediction'])
    predicted_dfs.append(predicted_df)
    
predicted_full = pd.concat(predicted_dfs)

nyc-tlc/trip data/yellow_tripdata_2019-01.csv
nyc-tlc/trip data/yellow_tripdata_2019-02.csv
nyc-tlc/trip data/yellow_tripdata_2019-03.csv
nyc-tlc/trip data/yellow_tripdata_2019-04.csv
nyc-tlc/trip data/yellow_tripdata_2019-05.csv
nyc-tlc/trip data/yellow_tripdata_2019-06.csv
nyc-tlc/trip data/yellow_tripdata_2019-07.csv
nyc-tlc/trip data/yellow_tripdata_2019-08.csv
nyc-tlc/trip data/yellow_tripdata_2019-09.csv
nyc-tlc/trip data/yellow_tripdata_2019-10.csv
nyc-tlc/trip data/yellow_tripdata_2019-11.csv
nyc-tlc/trip data/yellow_tripdata_2019-12.csv
CPU times: user 3min 24s, sys: 28.4 s, total: 3min 52s
Wall time: 8min


In [67]:
predicted_full.describe()

,prediction
count,8.419462e+07
mean,2.026640e-01
std,2.385823e-02
min,1.460668e-01
25%,1.821094e-01
50%,2.013440e-01
75%,2.237013e-01
max,2.564956e-01


--- 

## Train model with large dataset

We are stuck here, because we need to be able to load the full dataset into memory to train with it.\*

Think about all the data we're missing out on. All those models, all those hyperparameters that will never get a chance!

<img src="https://media0.giphy.com/media/k61nOBRRBMxva/giphy.gif" width="400" alt="crying" />

_\* Scikit-learn supports [incremental learning](https://scikit-learn.org/stable/modules/computing.html#incremental-learning) for models that have a `partial_fit` method. `ElasticNet` does not have a `partial_fit` method._

# There must be a better way!

<img src="https://docs.dask.org/en/latest/_images/dask_horizontal_no_pad.svg" width="300" alt="dask" />

With Dask, we can scale out to a cluster to address these four failure scenarios. 

Move on to [03-hyperparameter.ipynb](03-hyperparameter.ipynb) to get started!